In [2]:
# 4_proceso_etl.py
import pandas as pd
from pathlib import Path
from dateutil import parser
import numpy as np

# 1️⃣ Definir rutas
data_path = Path.cwd().parent / "data"
database_path = Path.cwd().parent / "database"
database_path.mkdir(parents=True, exist_ok=True)

input_file = data_path / "pacientes.csv"
output_file = database_path / "pacientes_clean.csv"

# 2️⃣ Leer el archivo CSV original
df = pd.read_csv(input_file)

# 3️⃣ Mostrar cantidad de valores nulos antes de la limpieza
print("🔍 Valores nulos antes de la limpieza:")
print(df.isnull().sum())
print(f"\nTotal de registros originales: {len(df)}")

# 4️⃣ Reemplazar valores nulos o 'NA' en columnas clave
df.replace(["NA", "N/A", ""], [np.nan, np.nan, np.nan], inplace=True)

# Rellenar valores vacíos
df["distrito"] = df["distrito"].fillna("Desconocido")
df["servicio"] = df["servicio"].fillna("Desconocido")
df["sexo"] = df["sexo"].fillna("ND")

# 5️⃣ Formatear fechas (intenta convertir distintos formatos)
def convertir_fecha(valor):
    try:
        return parser.parse(str(valor), dayfirst=False).strftime("%Y-%m-%d")
    except Exception:
        try:
            return parser.parse(str(valor), dayfirst=True).strftime("%Y-%m-%d")
        except Exception:
            return None

df["fecha_atencion"] = df["fecha_atencion"].apply(convertir_fecha)
df.dropna(subset=["fecha_atencion"], inplace=True)

# 6️⃣ Convertir tipos de datos numéricos y limpiar inconsistencias
df["edad"] = pd.to_numeric(df["edad"], errors="coerce")
df.loc[(df["edad"] < 0) | (df["edad"] > 120), "edad"] = np.nan

df["monto"] = pd.to_numeric(df["monto"], errors="coerce")

# 7️⃣ Eliminar filas completamente vacías o duplicadas
df.dropna(how="all", inplace=True)
df.drop_duplicates(inplace=True)

# 8️⃣ Mostrar resumen después de la limpieza
print("\n✅ Después de la limpieza:")
print(df.isnull().sum())
print(f"\nTotal de registros limpios: {len(df)}")

# 9️⃣ Guardar el nuevo archivo limpio
df.to_csv(output_file, index=False, encoding="utf-8")
print(f"\n📁 Archivo limpio guardado en: {output_file}")


🔍 Valores nulos antes de la limpieza:
id_paciente         0
nombre              0
edad               95
sexo              247
distrito          110
servicio          145
fecha_atencion      0
monto             116
dtype: int64

Total de registros originales: 1000

✅ Después de la limpieza:
id_paciente         0
nombre              0
edad               95
sexo                0
distrito            0
servicio            0
fecha_atencion      0
monto             116
dtype: int64

Total de registros limpios: 1000

📁 Archivo limpio guardado en: e:\SaludDigitalBD_E1\database\pacientes_clean.csv
